<a href="https://colab.research.google.com/github/mcash8/DVBS2X-DNN-Implementation/blob/main/Feed%20Forward%20NN/HPA%20Tests/HPA%20AWGN/HPA_Unmatch_Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import os
import scipy.io as spio
import math
import time
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import MeanSquaredError

In [ ]:
%cd /content/drive/My Drive/NN\ Input/HPA/AWGN/Train\ Data
start_time = time.time()

#parameters
num_classes = 2
batch_size = 64
epochs = 100

BO = [30]
SNR = [2,10,23]

for snr in SNR: 
    for bo in BO: 
        matname = 'DVBSAPSK_data_SNR_%d_BO_%d.mat' % (snr, bo)
        print(matname)
        mat = spio.loadmat(matname, squeeze_me=True)
        x_train = mat['x_train']
        x_valid = mat['val_data']
        y_valid = mat['val_target']
        y_train = mat['y_train']


        #cast to float32
        x_train = x_train.astype('float32')
        x_valid = x_valid.astype('float32')
        #x_test = x_test.astype('float32') 

        print(x_train.shape, 'train samples')
        print(x_valid.shape, 'valid samples')

        print(y_train.shape, 'train labels')
        print(y_valid.shape, 'valid labels')

        print('Label Examples:\n', y_train)
        print('Input Examples:\n', x_train)

        # Define the network - change input dim based on window size
        model = Sequential()
        model.add(Dense(50, activation='elu', input_dim=18))
        model.add(Dense(50, activation='elu'))
        model.add(Dense(num_classes, activation='tanh'))

        model.summary()

        model.compile(loss=keras.metrics.mean_squared_error,
                    optimizer=SGD(learning_rate=0.1),
                    metrics=[keras.metrics.RootMeanSquaredError(name='rmse')])

        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=2,
                            validation_data=(x_valid, y_valid))

        elapsedTime = (time.time() - start_time)/60
        print("--- %.2f minutes ---" % elapsedTime)

        savename = 'model_SNR_%d_BO_%d.h5' % (snr,bo)
        model.save(savename)

In [ ]:
%cd /content/drive/My Drive/NN\ Input/HPA/AWGN/Test\ Data/unmatch

BO = [1,7,30] #backoffs 
test_bos = [[7,30], [30,1], [1,7]]
train_snrs = [2, 10, 23] #training snrs
snrs = list(range(-2,24))  #snrs

start_time = time.time()

for train_snr in train_snrs: 
    %cd /content/drive/My Drive/NN\ Input/HPA/AWGN/Test\ Data/unmatch
    k=0
    #load trained model at set BO
    for bo in BO:
        #make folder for results - 
        foldername = 'BO_%d_trainSNR_%d' % (bo, train_snr) 
        %mkdir $foldername 

        #load train BO
        %cd /content/drive/My Drive/NN\ Input/HPA/AWGN/Train\ Data
        model_name = 'model_SNR_%d_BO_%d.h5' % (train_snr, bo)  #format model name
        model = tf.keras.models.load_model(model_name) #load trained model at set back off and snr
        #loop through data that's not the same bo as model
        for test_bo in test_bos[k]:
            #make subfolder in larger folder
            %cd /content/drive/My Drive/NN\ Input/HPA/AWGN/Test\ Data/unmatch
            %cd $foldername 
            subfolder = 'test_BO%d' % (test_bo)
            #make subfolder
            %mkdir $subfolder
            for snr in snrs:  
                #loop through snrs  
                for x in range(1,11):
                    #loop through packets
                    %cd /content/drive/My Drive/NN\ Input/HPA/AWGN/Test\ Data
                    folder = 'BO%d' % (test_bo)
                    %cd $folder
                    matname = 'DVBSAPSK_pred_bo_%d_snr_%d_%d' % (test_bo, snr, x)
                    print(matname)
                    mat = spio.loadmat(matname, squeeze_me=True) #load data
                    x_test = mat['train_data']
                    
                    predictions = model.predict(x_test) #make predictions
                    %cd /content/drive/My Drive/NN\ Input/HPA/AWGN/Test\ Data/unmatch
                    %cd $foldername 
                    %cd $subfolder

                    #save predictions 
                    matname = 'DVBSAPSK_out_bo_%d_snr_%d_%d_mdl_%d.mat' % (test_bo,snr,x, bo)
                    spio.savemat(matname, {'pred': predictions})

        %cd /content/drive/My Drive/NN\ Input/HPA/AWGN/Test\ Data/unmatch
        k+=1 #move to next unmatch pair
elapsedTime = (time.time() - start_time)/60
print("--- %.2f minutes ---" % elapsedTime)

/content/drive/My Drive/NN Input/HPA/AWGN/Test Data/unmatch
/content/drive/My Drive/NN Input/HPA/AWGN/Test Data/unmatch
/content/drive/My Drive/NN Input/HPA/AWGN/Train Data
/content/drive/My Drive/NN Input/HPA/AWGN/Test Data/unmatch
/content/drive/My Drive/NN Input/HPA/AWGN/Test Data/unmatch/BO_1_trainSNR_2
/content/drive/My Drive/NN Input/HPA/AWGN/Test Data
/content/drive/My Drive/NN Input/HPA/AWGN/Test Data/BO7
DVBSAPSK_pred_bo_7_snr_-2_1
/content/drive/My Drive/NN Input/HPA/AWGN/Test Data/unmatch
/content/drive/My Drive/NN Input/HPA/AWGN/Test Data/unmatch/BO_1_trainSNR_2
/content/drive/My Drive/NN Input/HPA/AWGN/Test Data/unmatch/BO_1_trainSNR_2/test_BO7
/content/drive/My Drive/NN Input/HPA/AWGN/Test Data
/content/drive/My Drive/NN Input/HPA/AWGN/Test Data/BO7
DVBSAPSK_pred_bo_7_snr_-2_2
/content/drive/My Drive/NN Input/HPA/AWGN/Test Data/unmatch
/content/drive/My Drive/NN Input/HPA/AWGN/Test Data/unmatch/BO_1_trainSNR_2
/content/drive/My Drive/NN Input/HPA/AWGN/Test Data/unmatch/BO

In [ ]:
%cd /content/drive/My Drive/NN\ Input/HPA/AWGN/Test\ Data/unmatch
BO = [1] #backoffs 
test_bos = [[7,30], [30,1], [1,7]]
train_snrs = [2, 10, 23] #training snrs
snrs = list(range(1,3))  #snrs

for train_snr in train_snrs: 
    %cd /content/drive/My Drive/NN\ Input/HPA/AWGN/Test\ Data/unmatch
    k=0
    for bo in BO:
        #make folder for results - 
        foldername = 'BO_%d_trainSNR_%d' % (bo, train_snr) 
        %mkdir $foldername 

        for test_bo in test_bos[k]:
            #make subfolder in larger folder
            %cd /content/drive/My Drive/NN\ Input/HPA/AWGN/Test\ Data/unmatch
            %cd $foldername 
            subfolder = 'test_BO%d' % (test_bo)
            #make subfolder
            %mkdir $subfolder

            for snr in snrs:
                for x in range(1):
                    %cd /content/drive/My Drive/NN\ Input/HPA/AWGN/Test\ Data/unmatch
                    %cd $foldername 
                    %cd $subfolder
                    #dummy mat file 
                    pred = 1
                    matname = 'test.mat'
                    spio.savemat(matname, {'pred': pred})

                    %cd /content/drive/My Drive/NN\ Input/HPA/AWGN/Test\ Data
        
    k+=1 #move to next unmatch pair